In [ ]:
#General data analysis and visualisation libraries:
import time
s=time.time()
import os
import pandas as pd
import numpy as np
from pyarrow import csv
import tables
import dask
import time
import h5py
from dask import dataframe as dd
import seaborn as sns
pd.options.display.max_rows = 999

# Dimensionality reduction libraries:
#PCA
from sklearn.decomposition import PCA
#TSNE
from sklearn.manifold import TSNE
#UMAP
import umap.umap_ as umap

# Machine learning libraries:
#prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#machine learning
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeClassifier , plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.svm import SVC

#evaluation
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import r2_score

import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
import scipy.linalg
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt



e=time.time()
print('imported in: {} seconds'.format(round(e-s,3)))

imported in: 9.162 seconds


Check for GPU:

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

<br><br>
### _Create high-m (long) data_
10m rows

In [ ]:
xlong = np.reshape(np.random.rand(10000000), (10000000,1))

In [ ]:
any(xlong<0), any(xlong>1)

In [ ]:
xlong.shape

_Target variable for long_

In [ ]:
ylong= (20*xlong) - 5 +  np.random.rand(10000000,1)/10
# y=20x -5 + noise


##### Hidden formula for xlong: y=20x-5

<br><br>
### _Create high-n (wide) data_

In [ ]:
np.random.rand(50,50)

In [ ]:
xwide = np.random.rand(3000,3000)

In [ ]:
i=1
temp=-5
for c in xwide.T:
    temp += (c*i)
    i+=1

In [ ]:
ywide=temp

In [ ]:
ywide.shape

Adding the ones:

In [ ]:
ones_long = np.c_[np.ones((10000000, 1)), xlong]
ones_wide = np.c_[np.ones((3000, 1)), xwide]

### Amex dataset (raw)

In [ ]:
os.chdir('E:\\Ice Drive\\data\\amex\\amex-default-prediction\\clean')
store = pd.HDFStore('train_grouped.hdf5')
start=time.time()
amex = store['data']
end=time.time()
print('Read time into pandas DataFrame was: {} seconds.'.format(end-start))

##### Load labels and join onto amex DataFrame

In [ ]:
os.chdir('E:\\Ice Drive\\data\\amex\\amex-default-prediction')
labels = pd.read_csv('train_labels.csv', index_col='customer_ID')
amex=amex.join(labels)

### Amex dataset (9PCs)

In [ ]:
s=time.time()
amex_mat=amex.to_numpy()
amex = amex.fillna(np.nanmean(amex_mat[:,:156]))
# PCA
pca = PCA(n_components=9)
principalComponents = pca.fit_transform(amex.iloc[:,:156])
#JOIN LABELS
amex_pca=pd.DataFrame(principalComponents, columns=['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9',], index=amex.index).join(labels)
t=time.time()

In [ ]:
t-s

#### Datasets:
* long -> 10 mn rows, 2 column
* wide -> 3k rows, 3k columns
* amex -> 458913 rows, 157 columns
* amex_pca -> 458913 rows, 10 columns

**All data now ready**
<br>
_m_ dataset is very long
<br>
_n_ data set is very wide

In [ ]:
X = xlong
y = ylong
# test/train
X_train, X_test, y_train, y_test =train_test_split(
                        X,y,
                        test_size=0.33, random_state=42)

</br>Sklearn test:

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
y_pred=reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as err
err(y_test, y_pred)

In [ ]:
reg.coef_, reg.intercept_

### Normal equation

In [ ]:
X = xlong
y = ylong
# test/train
X_train, X_test, y_train, y_test =train_test_split(
                        ones_long,y,
                        test_size=0.33, random_state=42)

#### Long

In [ ]:
%%timeit -n 1
np.linalg.inv(mb_train.T.dot(mb_train)).dot(mb_train.T).dot(ym_train)

In [ ]:
np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)

#### Wide

In [ ]:
%%timeit -n 1
np.linalg.inv(nb_train.T.dot(nb_train)).dot(nb_train.T).dot(yn_train)

### Moore-Penrose Pseudo inverse (using SVD)

#### Long

In [ ]:

np.linalg.lstsq(X_train,y_train)

#### Wide

In [ ]:
%%timeit -n 1
np.linalg.lstsq(nb_train,yn_train)[0]

#### Long

In [ ]:
%%timeit -n 1
scipy.linalg.lstsq(mb_train,ym_train)[0]

#### Wide

In [ ]:
%%timeit -n 1
scipy.linalg.lstsq(nb_train,yn_train)[0]

#### Long

In [ ]:
%%timeit -n 1
np.linalg.pinv(mb_train).dot(mb_train)

#### Wide Pinv

In [ ]:
%%timeit -n 1
np.linalg.pinv(nb_train).dot(yn_train)

#### long scikit-learn

In [ ]:
%%timeit -n 1
reg = LinearRegression().fit(m, ym)

#### Wide scikit-learn

In [ ]:
%%timeit -n 1
reg = LinearRegression().fit(n, yn)

### Gradient Descent (batch)

#### Wide

In [ ]:
s=time.time()
theta = np.random.randn(2,1) # random initialization
eta = 0.1 # learning rate
n_iterations = 1000
m = len(X_train)

for iteration in range(n_iterations):
    gradients = 2/m * X_train.T.dot(X_train.dot(theta) - y_train)
    theta = theta - eta * gradients

e=time.time()

In [ ]:
e-s

### Train/test split

In [ ]:
X = amex_pca.to_numpy()[:,:9]
y = amex_pca.to_numpy()[:,9:]

# test/train
X_train, X_test, y_train, y_test =train_test_split(
                        X,y,
                        test_size=0.33, random_state=42)

<br><br>
Decision tree on amex pca

In [ ]:
s=time.time()

# Fit model
dtpca=DecisionTreeClassifier()
dtpca.fit(X_train,y_train)

# predict:
yhat=dtpca.predict(X_test)

# Plotting the consusion matrix

confusion_matrix = metrics.confusion_matrix(y_test, yhat)

## Get metrics:
TN=confusion_matrix[0][0]
FP=confusion_matrix[0][1]
FN=confusion_matrix[1][0]
TP=confusion_matrix[1][1]


#print metrics
dtpca_recall = TP/(TP+FN)
dtpca_specificity = 1-(FP/(TN+FP))
dtpca_acc = (TP+TN)/(TP+FP+TN+FN)

print('Accuracy: ' + str(round(dtpca_acc,3)))
print('Specificity: ' + str(round(dtpca_specificity,3)))
print('Recall: ' + str(round(dtpca_recall,3)))

e=time.time()
print('Time taken: {} secs'.format(round((e-s),2)))

<br><br>
Random Forest on amex pca

In [ ]:
s3=time.time()
# fit
rfpca=RandomForestClassifier()
rfpca.fit(X_train,y_train)

# predict:
yhat=rfpca.predict(X_test)

# Creating the consusion matrix

confusion_matrix = metrics.confusion_matrix(y_test, yhat)

## Get metrics:
TN=confusion_matrix[0][0]
FP=confusion_matrix[0][1]
FN=confusion_matrix[1][0]
TP=confusion_matrix[1][1]


#print metrics
rfpca_recall = TP/(TP+FN)
rfpca_specificity = 1-(FP/(TN+FP))
rfpca_acc = (TP+TN)/(TP+FP+TN+FN)
print('Accuracy: ' + str(round(rfpca_acc,3)))
print('Specificity: ' + str(round(rfpca_specificity,3)))
print('Recall: ' + str(round(rfpca_recall,3)))
e3=time.time()
print('Time taken: {} mins'.format(round((e3-s3)/60,2)))

</br>MLP neural network on pca

In [ ]:
#Hyperparameters:
layers= 8
batch_size=48
nodes=40
activation = 'tanh'
epochs=30
initializer='random_normal'
optimizer='Adam'
learning_rate=0.001
regularizer='l2'

s=time.time()

model_name='illumidesk'
model = Sequential([
Dense(units=nodes, input_shape=(9,), kernel_initializer = initializer, activation=activation)], name='illumidesk')

for i in range(layers):
    model.add(keras.layers.Dense(units=nodes, activation=activation, kernel_regularizer=regularizer))

model.add(keras.layers.Dense(units=2,
                name="Output-Layer",
                activation='softmax'))

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history=model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2, validation_split=0.15,validation_data=None)

e=time.time()
times_do[model_name]=round(e-s,3)

In [ ]:
times_do

</br>MLP neural network on AMEX full

In [ ]:
X = amex.to_numpy()[:,:156]
y = amex.to_numpy()[:,156:]

# test/train
X_train, X_test, y_train, y_test =train_test_split(
                        X,y,
                        test_size=0.33, random_state=42)

In [ ]:
#Hyperparameters:
layers= 8
batch_size=48
nodes=40
activation = 'tanh'
epochs=30
initializer='random_normal'
optimizer='Adam'
learning_rate=0.001
regularizer='l2'

s=time.time()

model_name='illumidesk2'
model = Sequential([
Dense(units=nodes, input_shape=(156,), kernel_initializer = initializer, activation=activation)], name='illumidesk2')

for i in range(layers):
    model.add(keras.layers.Dense(units=nodes, activation=activation, kernel_regularizer=regularizer))

model.add(keras.layers.Dense(units=2,
                name="Output-Layer",
                activation='softmax'))

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history=model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=2, validation_split=0.15,validation_data=None)

e=time.time()
times_do[model_name]=round(e-s,3)

In [ ]:
times_do

In [ ]:
MLP neural network on long

In [ ]:
X = xlong
y = ylong

# test/train
X_train, X_test, y_train, y_test =train_test_split(
                        X,y,
                        test_size=0.33, random_state=42)

In [ ]:
from keras import metrics

Below takes too long on my machine -> over 2 mins per epoch.

In [ ]:
#Hyperparameters:
layers= 10
batch_size=60
nodes=32
activation = 'relu'
epochs=5
initializer='random_normal'
optimizer='Adam'
learning_rate=0.01
regularizer='l2'

s=time.time()

model_name='illumidesk_long'
model = Sequential([
Dense(units=nodes, input_shape=(1,), kernel_initializer = initializer, activation=activation)], name='illumidesk_long')

for i in range(layers):
    model.add(keras.layers.Dense(units=nodes, activation=activation, kernel_regularizer=regularizer))

model.add(keras.layers.Dense(units=2,
                name="Output-Layer",
                activation='relu'))

model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mean_squared_error", metrics=['mse'])

history=model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1, validation_split=0.15,validation_data=None)
e=time.time()

times_do[model_name]=round(e-s,3)